# 06 Context Engineering & Multi-Agent Orchestration

This notebook demonstrates advanced **LangGraph** patterns using the IBM Granite model.

We will build a **Multi-Agent System** with three specific nodes:
1. **Context Engineer**: Analyzes the input to determine the appropriate "tone" and "constraint" (Context Engineering).
2. **Supervisor**: Routes the task to the correct specialist based on the request.
3. **Specialist Agents**: 
    - `MetricAgent`: Handles data and counting tasks.
    - `CreativeAgent`: Handles writing and storytelling tasks.

In [1]:
import sys
import os
import operator
from typing import Annotated, List, TypedDict, Union

# Setup paths
sys.path.append(os.path.abspath('..'))

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END

load_dotenv()

# Initialize Model
llm = ChatOpenAI(
    model=os.getenv("MODEL_NAME", "ibm-granite/granite-4.0-h-micro"),
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0.1
)

### 1. Define State & Context
We define a `GraphState` that holds not just messages, but also specific **Context** variables that guide agent behavior.

In [2]:
class GraphState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    # Context variables injected by the Context Engineer
    tone: str
    constraints: str
    next_agent: str

### 2. Define The Nodes

#### Node A: Context Engineer
This node looks at the user input and decides *how* the agents should behave. This is "Context Engineering" — separating the *how* from the *what*.

In [3]:
def context_engineer_node(state: GraphState):
    messages = state['messages']
    last_message = messages[-1].content
    
    # Prompt to extract context
    prompt = f"""
    Analyze the following user request: "{last_message}"
    
    Determine the appropriate TONE (e.g., Professional, Pirate, Robotic, Poetic) and CONSTRAINTS (e.g., Concise, Verbose, JSON-only).
    
    Respond in this exact format:
    Tone: <tone>
    Constraints: <constraints>
    """
    
    response = llm.invoke(prompt).content
    
    # Simple parsing (robustness depends on model)
    tone = "Professional" # Default
    constraints = "None" # Default
    
    for line in response.split('\n'):
        if line.startswith("Tone:"):
            tone = line.split("Tone:")[1].strip()
        if line.startswith("Constraints:"):
            constraints = line.split("Constraints:")[1].strip()
            
    print(f"🔍 [Context Engineer] Detected Tone: {tone} | Constraints: {constraints}")
    return {"tone": tone, "constraints": constraints}

#### Node B: Supervisor (Router)
Decides which agent is best suited for the task.

In [4]:
def supervisor_node(state: GraphState):
    messages = state['messages']
    last_message = messages[-1].content
    
    prompt = f"""
    You are a Supervisor. You have two workers:
    1. MetricAgent: Good for counting, math, logic, and facts.
    2. CreativeAgent: Good for writing, stories, poems, and ideas.
    
    User Request: "{last_message}"
    
    Respond with ONLY the name of the agent to handle this: 'MetricAgent' or 'CreativeAgent'.
    """
    
    response = llm.invoke(prompt).content.strip()
    
    # Clean up response (remove punctuation/extra spaces)
    next_agent = response.replace("'", "").replace("\"", "").strip()
    
    # Fallback
    if "Metric" in next_agent: next_agent = "MetricAgent"
    elif "Creative" in next_agent: next_agent = "CreativeAgent"
    else: next_agent = "CreativeAgent" # Default
    
    print(f"👮 [Supervisor] Routing to: {next_agent}")
    return {"next_agent": next_agent}

#### Node C & D: The Workers
These agents consume the `tone` and `constraints` from the state to generate their answer.

In [5]:
def metric_agent_node(state: GraphState):
    messages = state['messages']
    tone = state['tone']
    constraints = state['constraints']
    
    system_prompt = f"""
    You are the MetricAgent. You focus on counting and hard facts.
    
    CONTEXT INSTRUCTIONS:
    - Adopt this Tone: {tone}
    - Follow Constraints: {constraints}
    """
    
    # We create a temporary message list for this generation
    prompt_messages = [SystemMessage(content=system_prompt)] + messages
    response = llm.invoke(prompt_messages)
    
    return {"messages": [response]}

def creative_agent_node(state: GraphState):
    messages = state['messages']
    tone = state['tone']
    constraints = state['constraints']
    
    system_prompt = f"""
    You are the CreativeAgent. You focus on storytelling and ideation.
    
    CONTEXT INSTRUCTIONS:
    - Adopt this Tone: {tone}
    - Follow Constraints: {constraints}
    """
    
    prompt_messages = [SystemMessage(content=system_prompt)] + messages
    response = llm.invoke(prompt_messages)
    
    return {"messages": [response]}

### 3. Build the Graph

In [6]:
workflow = StateGraph(GraphState)

# Add Nodes
workflow.add_node("context_engineer", context_engineer_node)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("metric_agent", metric_agent_node)
workflow.add_node("creative_agent", creative_agent_node)

# Set Entry Point
workflow.set_entry_point("context_engineer")

# Edges
# 1. Context Engineer -> Supervisor
workflow.add_edge("context_engineer", "supervisor")

# 2. Supervisor -> Specific Agent (Conditional)
def route_agents(state):
    if state['next_agent'] == "MetricAgent":
        return "metric_agent"
    else:
        return "creative_agent"

workflow.add_conditional_edges(
    "supervisor",
    route_agents,
    {
        "metric_agent": "metric_agent",
        "creative_agent": "creative_agent"
    }
)

# 3. Agents -> END
workflow.add_edge("metric_agent", END)
workflow.add_edge("creative_agent", END)

# Compile
app = workflow.compile()

### 4. Run the Simulation

In [7]:
def run_query(query):
    print(f"\n🚀 User Query: \"{query}\"")
    print("--------------------------------------------------")
    
    inputs = {"messages": [HumanMessage(content=query)]}
    
    # Run the graph
    result = app.invoke(inputs)
    
    final_msg = result['messages'][-1].content
    print("\n🤖 Final Output:")
    print(final_msg)

#### Scenario A: Logic + Pirate Tone

In [8]:
run_query("Count the number of characters in the word 'Shipwreck' but say it like a pirate.")


🚀 User Query: "Count the number of characters in the word 'Shipwreck' but say it like a pirate."
--------------------------------------------------
🔍 [Context Engineer] Detected Tone: Pirate | Constraints: Concise
👮 [Supervisor] Routing to: MetricAgent

🤖 Final Output:
Arr matey, the word 'Shipwreck' be holdin' 10 characters, me hearty!


#### Scenario B: Creative + Robotic Tone

In [9]:
run_query("Write a very short poem about silicon chips, but keep it robotic and concise.")


🚀 User Query: "Write a very short poem about silicon chips, but keep it robotic and concise."
--------------------------------------------------
🔍 [Context Engineer] Detected Tone: Robotic | Constraints: Concise
👮 [Supervisor] Routing to: MetricAgent

🤖 Final Output:
Silicon, precise, in circuits we trust.
